In [1]:
import duckdb
from packages import constants

In [2]:
conn = duckdb.connect(constants.DATABASE_NAME)

In [3]:
query = """DROP TABLE IF EXISTS users"""
conn.execute(query)
query = """DROP TABLE IF EXISTS logon_events"""
conn.execute(query)

In [4]:
query = """
CREATE TABLE IF NOT EXISTS users (
    userid VARCHAR(50) PRIMARY KEY,
    created_at TIMESTAMP
)
"""
conn.execute(query)

In [5]:
query = """
CREATE TABLE IF NOT EXISTS logon_events (
    userid VARCHAR(50),
    logon_ts TIMESTAMP
)
"""
conn.execute(query)

In [6]:
query = """SELECT * FROM logon_events"""
conn.execute(query).df()

,userid,logon_ts


In [7]:
query = """
INSERT INTO users (userid, created_at)
SELECT
    CONCAT('user', number['generate_series']),
    get_current_timestamp() - (CAST(RANDOM() * 2592000 AS INTEGER) || ' SECONDS')::INTERVAL
FROM GENERATE_SERIES(1, 10) AS number; 
"""
conn.execute(query)

In [8]:
query = """
INSERT INTO logon_events (userid, logon_ts)
SELECT userid, created_at + INTERVAL '1 DAY' AS logon_date
FROM users
WHERE userid in ('user2', 'user4', 'user3', 'user10');
"""
conn.execute(query)

In [9]:
query = """
INSERT INTO logon_events (userid, logon_ts)
SELECT userid, created_at + INTERVAL '2 DAY' AS logon_date
FROM users
WHERE userid in ('user1', 'user5', 'user4');

INSERT INTO logon_events (userid, logon_ts)
SELECT userid, created_at + INTERVAL '51 HOURS' AS logon_date
FROM users
WHERE userid in ('user5', 'user4');

INSERT INTO logon_events (userid, logon_ts)
SELECT userid, created_at + INTERVAL '3 DAY' AS logon_date
FROM users
WHERE userid in ('user1', 'user10', 'user9', 'user5', 'user6');
"""
conn.execute(query)

In [10]:
query = """SELECT * FROM users"""
conn.execute(query).df()

,userid,created_at
0,user1,2023-12-08 23:18:37.722
1,user2,2023-11-27 17:21:01.722
2,user3,2023-12-17 11:08:53.722
3,user4,2023-12-05 11:27:57.722
4,user5,2023-12-07 16:45:56.722
5,user6,2023-12-17 15:36:16.722
6,user7,2023-12-08 17:52:47.722
7,user8,2023-12-10 00:24:49.722
8,user9,2023-12-02 01:11:58.722
9,user10,2023-12-15 20:06:11.722


In [11]:
query = """SELECT * FROM logon_events"""
conn.execute(query).df()

,userid,logon_ts
0,user2,2023-11-28 17:21:01.722
1,user4,2023-12-06 11:27:57.722
2,user3,2023-12-18 11:08:53.722
3,user10,2023-12-16 20:06:11.722
4,user1,2023-12-10 23:18:37.722
5,user5,2023-12-09 16:45:56.722
6,user4,2023-12-07 11:27:57.722
7,user5,2023-12-09 19:45:56.722
8,user4,2023-12-07 14:27:57.722
9,user1,2023-12-11 23:18:37.722


In [12]:
conn.close()